In [ ]:
Pkg.update()

In [ ]:
## EXAMPLE: basic constrained problem

using JuMP
using Gurobi

m = Model(solver=GurobiSolver())

@variable(m, 0 <= x <= 2)
@variable(m, 0 <= y <= 30)

@objective(m, Max, 5x + 3y)
@constraint(m, 1x + 5y <= 3.0)

print(m)

status = solve(m)

println("Objective value: ", getobjectivevalue(m))
println("x = ", getvalue(x))
println("y = ", getvalue(y))

In [ ]:
## EXAMPLE: matrix multiplication

A = [1 2 3; 4 5 6]
x_actual = [3; 1; 4]
x_current = [1; 4; 1]
b = [5; 9]
y_actual = A*x_actual + b

using JuMP
using Gurobi

m1 = Model(solver=GurobiSolver())

@variable(m1, e[1:3])
@variable(m1, x[1:3])
@variable(m1, y[1:2])
@constraint(m1, x .== x_current) # input
@constraint(m1, y .== y_actual) # output constraints
@constraint(m1, A*(x+e) + b .== y)

@objective(m1, Min, sum(e.^2))

print(m1)

status = solve(m1)

println("Objective value: ", getobjectivevalue(m1))
# TODO: Are jump solutions global? Can I save particular variables?
println("e = ", getvalue(e))

In [ ]:
### Exploring convolutions and cross-correlations
# Looks like built-in convolution operation doesn't really meet our needs

image = [
    1 2 3 4 5;
    2 3 4 5 6;
    3 4 5 6 7;
    4 5 6 7 8;
    5 6 7 8 9
]

filter = [
    -3 -1 1;
    0 0 0;
    3 1 -1;
]

filter_flipped = filter[end:-1:1, end:-1:1]

conv2(image, filter_flipped)

In [36]:
## Example: convolution operation

include("nn_ops.jl")

using JuMP
using Gurobi

batch = 1
in_height = 3
in_width = 7
in_channels = 1
filter_height = 2
filter_width = 4
out_channels = 1

srand(1)
x_actual = rand(1:10, batch, in_height, in_width, in_channels)
x_current = rand(1:10, batch, in_height, in_width, in_channels)
filter = rand(1:10, filter_height, filter_width, in_channels, out_channels)
y_actual = conv2d(x_actual, filter)

m2 = Model(solver=GurobiSolver())

@variable(m2, e[1:batch, 1:in_height, 1:in_width, 1:in_channels])
@variable(m2, x[1:batch, 1:in_height, 1:in_width, 1:in_channels])
@variable(m2, y[1:batch, 1:in_height, 1:in_width, 1:out_channels])
@constraint(m2, x .== x_current) # input
@constraint(m2, y .== y_actual) # output constraints
@constraint(m2, conv2d(x+e, filter) .== y)

@objective(m2, Min, sum(e.^2))

print(m2)

status = solve(m2)

println("Objective value: ", getobjectivevalue(m2))
# TODO: Are jump solutions global? Can I save particular variables?
println("e = ", getvalue(e))

Min e[1,1,1,1]² + e[1,1,2,1]² + e[1,1,3,1]² + e[1,1,4,1]² + e[1,1,5,1]² + e[1,1,6,1]² + e[1,1,7,1]² + e[1,2,1,1]² + e[1,2,2,1]² + e[1,2,3,1]² + e[1,2,4,1]² + e[1,2,5,1]² + e[1,2,6,1]² + e[1,2,7,1]² + e[1,3,1,1]² + e[1,3,2,1]² + e[1,3,3,1]² + e[1,3,4,1]² + e[1,3,5,1]² + e[1,3,6,1]² + e[1,3,7,1]²
Subject to
 x[1,1,1,1] = 10
 x[1,2,1,1] = 2
 x[1,3,1,1] = 5
 x[1,1,2,1] = 5
 x[1,2,2,1] = 3
 x[1,3,2,1] = 8
 x[1,1,3,1] = 7
 x[1,2,3,1] = 6
 x[1,3,3,1] = 4
 x[1,1,4,1] = 6
 x[1,2,4,1] = 10
 x[1,3,4,1] = 4
 x[1,1,5,1] = 2
 x[1,2,5,1] = 4
 x[1,3,5,1] = 9
 x[1,1,6,1] = 1
 x[1,2,6,1] = 1
 x[1,3,6,1] = 7
 x[1,1,7,1] = 4
 x[1,2,7,1] = 8
 x[1,3,7,1] = 2
 y[1,1,1,1] = 209
 y[1,2,1,1] = 264
 y[1,3,1,1] = 134
 y[1,1,2,1] = 177
 y[1,2,2,1] = 119
 y[1,3,2,1] = 43
 y[1,1,3,1] = 175
 y[1,2,3,1] = 188
 y[1,3,3,1] = 87
 y[1,1,4,1] = 135
 y[1,2,4,1] = 171
 y[1,3,4,1] = 84
 y[1,1,5,1] = 181
 y[1,2,5,1] = 159
 y[1,3,5,1] = 66
 y[1,1,6,1] = 107
 y[1,2,6,1] = 140
 y[1,3,6,1] = 67
 y[1,1,7,1] = 92
 y[1,2,7,1] = 85
 y

410.0
e = [-1.0 5.0 5.0]

[5.0 1.0 -7.0]

[-4.0 -2.0 5.0]

[-4.0 -9.0 -3.0]

[3.0 1.0 -6.0]

[3.0 4.0 1.0]

[4.0 -6.0 3.0]


In [ ]:
## Example: rectified linearity

include("nn_ops.jl")

using JuMP
using Gurobi

batch = 1
in_height = 3
in_width = 7
in_channels = 1

srand(1)
x_current = rand(-10:10, batch, in_height, in_width, in_channels)

m3 = Model(solver=GurobiSolver())

@variable(m3, x[1:batch, 1:in_height, 1:in_width, 1:in_channels])
@variable(m3, e[1:batch, 1:in_height, 1:in_width, 1:in_channels])
@variable(m3, y[1:batch, 1:in_height, 1:in_width, 1:in_channels])
reluconstraint(m3, x+e, y, 1000)

@constraint(m3, x .== x_current) # input

@constraint(m3, y .== 0)

@objective(m3, Min, sum((x+e).^2))

print(m3)

status = solve(m3)

println("Objective value: ", getobjectivevalue(m3))
println("x = ", getvalue(x))
println("e = ", getvalue(e))

In [ ]:
## Example: max pooling

include("nn_ops.jl")

using JuMP
using Gurobi
using Base.Cartesian

batch = 1
in_height = 5 # TODO: test for non-matching in and pool heights
in_width = 10
in_channels = 1
pool_height = 3
pool_width = 2

out_height = round(Int, in_height/pool_height, RoundUp)
out_width = round(Int, in_width/pool_width, RoundUp)

srand(1)
x_current = rand(-10:10, batch, in_height, in_width, in_channels)

m4 = Model(solver=GurobiSolver())

@variable(m4, x[1:batch, 1:in_height, 1:in_width, 1:in_channels])
@variable(m4, y[1:batch, 1:out_height, 1:out_width, 1:in_channels])
maxpoolconstraint(m4, x, y, (3, 2) , 1000)

@constraint(m4, x .== x_current) # input

@objective(m4, Min, sum(y.^2))

print(m4)

status = solve(m4)

println("Objective value: ", getobjectivevalue(m4))
println("x = ", getvalue(x))
println("y = ", getvalue(y))

In [ ]:
x=(1,2)

In [ ]:
image[1, 2]

In [ ]:
image[x]

In [ ]:
image[1:3, 1:5]

In [ ]:
x=(1:3, 1:5)
image[x[1], x[2]]

In [ ]:
ind2sub(image, 3)

In [ ]:
maximum(getindex(image, x[1], x[2]))

In [ ]:
x_pool[1, 1] = 2

In [ ]:
it = zip(1:5, 11:15)

In [ ]:
for e in it
    println(e)
end
    

In [1]:
# making some fake array
A_target_size=(3,10,10, 7)
A = reshape(1:prod(A_target_size), A_target_size...)

strides = (1, 2, 2, 1)
child_index = (2, 1, 4, 1)



(2,1,4,1)

In [30]:
function getsliceindex(input_array_size::Int, stride::Int, output_index::Int)::AbstractArray{Int, 1}
    parent_start_index = (output_index-1)*stride+1
    parent_end_index = min((output_index)*stride, input_array_size)
    if parent_start_index > parent_end_index
        return []
    else
        return parent_start_index:parent_end_index
    end
end

"""
For pooling operations on an array, returns a view of the parent array
"""
function getpoolview{T<:Any, N}(input_array::AbstractArray{T, N}, strides::NTuple{N, Int}, output_index::NTuple{N, Int})::SubArray{T, N}
    it = zip(size(input_array), strides, output_index)
    input_index_range = map((x)-> getsliceindex(x ...), it)
    return view(input_array, input_index_range...)
end

function poolmap{T<:Any, N}(f::Function, input_array::AbstractArray{T, N}, strides::NTuple{N, Int})
    output_size::NTuple{N, Int} = ((x, y) -> round(Int, x/y, RoundUp)).(size(input_array), strides)
    output_indices = collect(CartesianRange(output_size))
    output_array::AbstractArray{T, N} = ((I) -> f(getpoolview(A, strides, I.I))).(output_indices)
    return output_array
end

poolmap (generic function with 2 methods)

In [31]:
getpoolview(A, strides, child_index)

1×2×2×1 SubArray{Int64,4,Base.ReshapedArray{Int64,4,UnitRange{Int64},Tuple{}},Tuple{UnitRange{Int64},UnitRange{Int64},UnitRange{Int64},UnitRange{Int64}},false}:
[:, :, 1, 1] =
 182  185

[:, :, 2, 1] =
 212  215

In [33]:
poolmap(maximum, A, strides)

3×5×5×7 Array{Int64,4}:
[:, :, 1, 1] =
 34  40  46  52  58
 35  41  47  53  59
 36  42  48  54  60

[:, :, 2, 1] =
 94  100  106  112  118
 95  101  107  113  119
 96  102  108  114  120

[:, :, 3, 1] =
 154  160  166  172  178
 155  161  167  173  179
 156  162  168  174  180

[:, :, 4, 1] =
 214  220  226  232  238
 215  221  227  233  239
 216  222  228  234  240

[:, :, 5, 1] =
 274  280  286  292  298
 275  281  287  293  299
 276  282  288  294  300

[:, :, 1, 2] =
 334  340  346  352  358
 335  341  347  353  359
 336  342  348  354  360

[:, :, 2, 2] =
 394  400  406  412  418
 395  401  407  413  419
 396  402  408  414  420

[:, :, 3, 2] =
 454  460  466  472  478
 455  461  467  473  479
 456  462  468  474  480

[:, :, 4, 2] =
 514  520  526  532  538
 515  521  527  533  539
 516  522  528  534  540

[:, :, 5, 2] =
 574  580  586  592  598
 575  581  587  593  599
 576  582  588  594  600

[:, :, 1, 3] =
 634  640  646  652  658
 635  641  647  653  659
 636  642  648  654

In [18]:
Union(typeof([]), typeof(1:5))

LoadError: MethodError: no method matching Union(::Type{Array{Any,1}}, ::Type{UnitRange{Int64}})[0m
Closest candidates are:
  Union{T}(::Any) at sysimg.jl:53[0m

In [32]:
f(x, y) = x+y

f (generic function with 1 method)

In [34]:
function twodo(f::Function{Int->Int}, x::Int, y::Int)
    return f(x, y)
end


LoadError: too many parameters for type Function

In [35]:
subtypes(Function)

LoadError: InterruptException: